# 1

The error arises because of a mismatch between the number of columns you are passing when constructing the DataFrame and the actual number of columns in the data. Specifically, in the `monthly_ledger` list, the number of elements in each entry exceeds the number of columns defined when creating the DataFrame. This mismatch leads to the `AssertionError`.

Let’s break it down and fix the issue.

### Problem:
You are appending the date and daily ledger to the `monthly_ledger` as a single list, which results in a structure that is difficult for pandas to parse correctly.

Here’s the line that causes the issue:
```python
all_ledgers.append([date] + daily_ledger)
```

This appends a date and the entire `daily_ledger` (which is already a list of lists) into a single list. As a result, pandas tries to interpret this as multiple columns, causing the mismatch.

### Solution:
You should process the `daily_ledger` and append each entry separately, ensuring each row has the correct number of columns when adding it to the `all_ledgers` list.

Here’s how you can fix it:

1. For each entry in `daily_ledger`, append the date alongside the transaction details.
2. Ensure that the resulting structure passed to pandas is a list of lists where each sublist has the correct number of columns.

Let’s modify the code:

```python
import pandas as pd
import random
import sqlite3
from datetime import datetime, timedelta

# Initialize the bank's starting balances
def initialize_balances():
    return {
        'Customer Deposits': 5000000,
        'Loans Outstanding': 2000000,
        'Cash Reserves': 1000000,
        'NOSTRO Account': 500000
    }

# Daily transactions simulation
def simulate_day(balances):
    # Initialize daily ledger
    daily_ledger = []
    
    # Randomize deposit, loans, payments, and treasury transactions
    deposit = random.randint(10000, 100000)
    loan_disbursed = random.randint(50000, 250000)
    payment_processed = random.randint(5000, 30000)
    treasury_transaction = random.randint(50000, 150000)
    
    # Deposit transaction
    balances['Customer Deposits'] += deposit
    daily_ledger.append(['Deposit', deposit, balances['Customer Deposits']])
    
    # Loan disbursed
    if balances['Cash Reserves'] >= loan_disbursed:
        balances['Loans Outstanding'] += loan_disbursed
        balances['Cash Reserves'] -= loan_disbursed
        daily_ledger.append(['Loan Disbursed', loan_disbursed, balances['Loans Outstanding']])
    
    # Payment processed for clients
    if balances['Customer Deposits'] >= payment_processed:
        balances['Customer Deposits'] -= payment_processed
        daily_ledger.append(['Client Payment', payment_processed, balances['Customer Deposits']])
    
    # Treasury transaction (NOSTRO account management)
    if balances['NOSTRO Account'] >= treasury_transaction:
        balances['NOSTRO Account'] -= treasury_transaction
        balances['Cash Reserves'] += treasury_transaction
        daily_ledger.append(['Treasury Transaction', treasury_transaction, balances['NOSTRO Account']])
    
    # Calculate daily profit
    daily_interest_income = random.randint(5000, 15000)  # Interest income on loans
    balances['Cash Reserves'] += daily_interest_income
    daily_ledger.append(['Interest Income', daily_interest_income, balances['Cash Reserves']])
    
    return daily_ledger

# Generate the daily financial report
def generate_daily_report(balances):
    balance_sheet = {
        'Assets': {
            'Loans Outstanding': balances['Loans Outstanding'],
            'Cash Reserves': balances['Cash Reserves'],
            'NOSTRO Account': balances['NOSTRO Account'],
            'Total Assets': balances['Loans Outstanding'] + balances['Cash Reserves'] + balances['NOSTRO Account']
        },
        'Liabilities': {
            'Customer Deposits': balances['Customer Deposits'],
            'Total Liabilities': balances['Customer Deposits']
        },
        'Equity': {
            'Total Equity': balances['Customer Deposits'] + balances['Loans Outstanding'] + balances['Cash Reserves'] + balances['NOSTRO Account']
        }
    }
    return balance_sheet

# Save the ledger to SQLite database and CSV file
def save_ledger_to_db_and_csv(ledger, db_name="bank_ledger.db", csv_name="bank_ledger.csv"):
    df = pd.DataFrame(ledger, columns=['Date', 'Transaction Type', 'Amount', 'Balance After'])
    conn = sqlite3.connect(db_name)
    df.to_sql('ledger', conn, if_exists='replace', index=False)
    df.to_csv(csv_name, index=False)
    conn.close()

# Simulate a month of operations
def simulate_month(balances):
    all_ledgers = []
    
    # Start date for the month simulation
    start_date = datetime.now()
    
    # Run daily simulation for 30 days
    for day in range(30):
        date = (start_date + timedelta(days=day)).strftime('%Y-%m-%d')
        daily_ledger = simulate_day(balances)
        
        # Append each transaction with the date to all_ledgers
        for transaction in daily_ledger:
            all_ledgers.append([date] + transaction)
        
        # Generate and print the daily financial report
        daily_report = generate_daily_report(balances)
        print(f"Day {day+1} Financial Report: {date}")
        print(pd.DataFrame(daily_report).T)
    
    return all_ledgers

# Main execution function
def main():
    balances = initialize_balances()
    
    # Simulate for one day
    print("\nSimulating one day of banking operations...")
    one_day_ledger = simulate_day(balances)
    print(one_day_ledger)
    
    # Simulate for a month
    print("\nSimulating one month of banking operations...")
    monthly_ledger = simulate_month(balances)
    
    # Save the ledger to database and CSV file
    print("\nSaving ledger to database and CSV...")
    save_ledger_to_db_and_csv(monthly_ledger)
    print("Ledger saved successfully!")

if __name__ == '__main__':
    main()
```

### Key Changes:
1. **Appending daily transactions**: For each transaction in the daily ledger, we append the date and transaction details individually to the `all_ledgers` list to ensure proper structure.
   
   ```python
   for transaction in daily_ledger:
       all_ledgers.append([date] + transaction)
   ```

2. **Fixing DataFrame Creation**: When creating the DataFrame, it now correctly expects four columns: `Date`, `Transaction Type`, `Amount`, and `Balance After`.

### Expected Output:
- **Ledger for the day and month**: All daily transactions are logged in the `all_ledgers` list.
- **Daily Financial Reports**: Printed as part of the loop during the simulation.
- **Database and CSV Output**: The ledger will be saved in both SQLite (`bank_ledger.db`) and CSV format (`bank_ledger.csv`) after the simulation.

This should resolve the column mismatch error and ensure that the ledger is structured correctly when saved.

You're absolutely correct. To follow proper accounting rules, we should mark transactions as either "debit" or "credit." Here's how the accounting treatment typically works:

- **Debit** (Dr) increases:
  - **Assets**: Cash, loans (from the bank's perspective, loans are assets because they generate income)
- **Credit** (Cr) decreases:
  - **Assets**: Cash outflows (e.g., client withdrawals)
- **Debit** (Dr) decreases:
  - **Liabilities**: Customer deposits decrease when customers withdraw funds
- **Credit** (Cr) increases:
  - **Liabilities**: Customer deposits increase when customers deposit funds

Let’s modify the code to apply the appropriate debit (Dr) and credit (Cr) tags to each transaction.

### Updated Python Code with Debit and Credit Marks:

```python
import pandas as pd
import random
import sqlite3
from datetime import datetime, timedelta

# Initialize the bank's starting balances
def initialize_balances():
    return {
        'Customer Deposits': 5000000,
        'Loans Outstanding': 2000000,
        'Cash Reserves': 1000000,
        'NOSTRO Account': 500000
    }

# Daily transactions simulation with debit and credit tags
def simulate_day(balances):
    # Initialize daily ledger
    daily_ledger = []
    
    # Randomize deposit, loans, payments, and treasury transactions
    deposit = random.randint(10000, 100000)
    loan_disbursed = random.randint(50000, 250000)
    payment_processed = random.randint(5000, 30000)
    treasury_transaction = random.randint(50000, 150000)
    
    # Deposit transaction (increase Customer Deposits - Cr, increase Cash Reserves - Dr)
    balances['Customer Deposits'] += deposit
    balances['Cash Reserves'] += deposit
    daily_ledger.append(['Deposit', 'Credit', deposit, 'Customer Deposits', balances['Customer Deposits']])
    daily_ledger.append(['Deposit', 'Debit', deposit, 'Cash Reserves', balances['Cash Reserves']])
    
    # Loan disbursed (increase Loans Outstanding - Dr, decrease Cash Reserves - Cr)
    if balances['Cash Reserves'] >= loan_disbursed:
        balances['Loans Outstanding'] += loan_disbursed
        balances['Cash Reserves'] -= loan_disbursed
        daily_ledger.append(['Loan Disbursed', 'Debit', loan_disbursed, 'Loans Outstanding', balances['Loans Outstanding']])
        daily_ledger.append(['Loan Disbursed', 'Credit', loan_disbursed, 'Cash Reserves', balances['Cash Reserves']])
    
    # Payment processed for clients (decrease Customer Deposits - Dr, decrease Cash Reserves - Cr)
    if balances['Customer Deposits'] >= payment_processed:
        balances['Customer Deposits'] -= payment_processed
        balances['Cash Reserves'] -= payment_processed
        daily_ledger.append(['Client Payment', 'Debit', payment_processed, 'Customer Deposits', balances['Customer Deposits']])
        daily_ledger.append(['Client Payment', 'Credit', payment_processed, 'Cash Reserves', balances['Cash Reserves']])
    
    # Treasury transaction (NOSTRO account decrease - Cr, Cash Reserves increase - Dr)
    if balances['NOSTRO Account'] >= treasury_transaction:
        balances['NOSTRO Account'] -= treasury_transaction
        balances['Cash Reserves'] += treasury_transaction
        daily_ledger.append(['Treasury Transaction', 'Credit', treasury_transaction, 'NOSTRO Account', balances['NOSTRO Account']])
        daily_ledger.append(['Treasury Transaction', 'Debit', treasury_transaction, 'Cash Reserves', balances['Cash Reserves']])
    
    # Calculate daily interest income (increase Cash Reserves - Dr, increase Interest Income - Cr)
    daily_interest_income = random.randint(5000, 15000)  # Interest income on loans
    balances['Cash Reserves'] += daily_interest_income
    daily_ledger.append(['Interest Income', 'Debit', daily_interest_income, 'Cash Reserves', balances['Cash Reserves']])
    daily_ledger.append(['Interest Income', 'Credit', daily_interest_income, 'Interest Income', None])
    
    return daily_ledger

# Generate the daily financial report
def generate_daily_report(balances):
    balance_sheet = {
        'Assets': {
            'Loans Outstanding': balances['Loans Outstanding'],
            'Cash Reserves': balances['Cash Reserves'],
            'NOSTRO Account': balances['NOSTRO Account'],
            'Total Assets': balances['Loans Outstanding'] + balances['Cash Reserves'] + balances['NOSTRO Account']
        },
        'Liabilities': {
            'Customer Deposits': balances['Customer Deposits'],
            'Total Liabilities': balances['Customer Deposits']
        },
        'Equity': {
            'Total Equity': balances['Customer Deposits'] + balances['Loans Outstanding'] + balances['Cash Reserves'] + balances['NOSTRO Account']
        }
    }
    return balance_sheet

# Save the ledger to SQLite database and CSV file
def save_ledger_to_db_and_csv(ledger, db_name="bank_ledger.db", csv_name="bank_ledger.csv"):
    df = pd.DataFrame(ledger, columns=['Transaction Type', 'Dr/Cr', 'Amount', 'Account', 'Balance After'])
    conn = sqlite3.connect(db_name)
    df.to_sql('ledger', conn, if_exists='replace', index=False)
    df.to_csv(csv_name, index=False)
    conn.close()

# Simulate a month of operations
def simulate_month(balances):
    all_ledgers = []
    
    # Start date for the month simulation
    start_date = datetime.now()
    
    # Run daily simulation for 30 days
    for day in range(30):
        date = (start_date + timedelta(days=day)).strftime('%Y-%m-%d')
        daily_ledger = simulate_day(balances)
        
        # Append each transaction with the date to all_ledgers
        for transaction in daily_ledger:
            all_ledgers.append([date] + transaction)
        
        # Generate and print the daily financial report
        daily_report = generate_daily_report(balances)
        print(f"Day {day+1} Financial Report: {date}")
        print(pd.DataFrame(daily_report).T)
    
    return all_ledgers

# Main execution function
def main():
    balances = initialize_balances()
    
    # Simulate for one day
    print("\nSimulating one day of banking operations...")
    one_day_ledger = simulate_day(balances)
    print(pd.DataFrame(one_day_ledger, columns=['Transaction Type', 'Dr/Cr', 'Amount', 'Account', 'Balance After']))
    
    # Simulate for a month
    print("\nSimulating one month of banking operations...")
    monthly_ledger = simulate_month(balances)
    
    # Save the ledger to database and CSV file
    print("\nSaving ledger to database and CSV...")
    save_ledger_to_db_and_csv(monthly_ledger)
    print("Ledger saved successfully!")

if __name__ == '__main__':
    main()
```

### Key Changes:
1. **Debit and Credit Tags**: Every transaction is marked as either a `Debit` or `Credit`:
    - Deposits are recorded as a **Credit** to the `Customer Deposits` account and a **Debit** to the `Cash Reserves` account.
    - Loans are recorded as a **Debit** to `Loans Outstanding` (asset) and a **Credit** to `Cash Reserves`.
    - Payments and treasury transactions follow similar debit/credit logic.

2. **Ledger Structure**: The `daily_ledger` now tracks:
    - Transaction Type (e.g., Deposit, Loan, Payment).
    - Debit/Credit (Dr/Cr).
    - Amount.
    - Account impacted (e.g., `Customer Deposits`, `Loans Outstanding`).
    - Balance after the transaction.

3. **Report Generation**: You can view daily transactions and financial reports as the simulation progresses.

### Output:
- **Ledger**: The output will now show the `Debit` and `Credit` markings for each transaction in line with double-entry bookkeeping.
- **Daily Financial Reports**: These reports display the current state of the bank’s assets, liabilities, and equity after each day’s transactions.
- **Database and CSV**: The ledger will be saved in both an SQLite database (`bank_ledger.db`) and a CSV file (`bank_ledger.csv`).

This approach ensures adherence to accounting standards and provides a clear view of the daily and monthly flow of financial transactions.

# 1 code

In [2]:
import pandas as pd
import random
import sqlite3
from datetime import datetime, timedelta

# Initialize the bank's starting balances
def initialize_balances():
    return {
        'Customer Deposits': 5000000,
        'Loans Outstanding': 2000000,
        'Cash Reserves': 1000000,
        'NOSTRO Account': 500000
    }

# Daily transactions simulation
def simulate_day(balances):
    # Initialize daily ledger
    daily_ledger = []
    
    # Randomize deposit, loans, payments, and treasury transactions
    deposit = random.randint(10000, 100000)
    loan_disbursed = random.randint(50000, 250000)
    payment_processed = random.randint(5000, 30000)
    treasury_transaction = random.randint(50000, 150000)
    
    # Deposit transaction
    balances['Customer Deposits'] += deposit
    daily_ledger.append(['Deposit', deposit, balances['Customer Deposits']])
    
    # Loan disbursed
    if balances['Cash Reserves'] >= loan_disbursed:
        balances['Loans Outstanding'] += loan_disbursed
        balances['Cash Reserves'] -= loan_disbursed
        daily_ledger.append(['Loan Disbursed', loan_disbursed, balances['Loans Outstanding']])
    
    # Payment processed for clients
    if balances['Customer Deposits'] >= payment_processed:
        balances['Customer Deposits'] -= payment_processed
        daily_ledger.append(['Client Payment', payment_processed, balances['Customer Deposits']])
    
    # Treasury transaction (NOSTRO account management)
    if balances['NOSTRO Account'] >= treasury_transaction:
        balances['NOSTRO Account'] -= treasury_transaction
        balances['Cash Reserves'] += treasury_transaction
        daily_ledger.append(['Treasury Transaction', treasury_transaction, balances['NOSTRO Account']])
    
    # Calculate daily profit
    daily_interest_income = random.randint(5000, 15000)  # Interest income on loans
    balances['Cash Reserves'] += daily_interest_income
    daily_ledger.append(['Interest Income', daily_interest_income, balances['Cash Reserves']])
    
    return daily_ledger

# Generate the daily financial report
def generate_daily_report(balances):
    balance_sheet = {
        'Assets': {
            'Loans Outstanding': balances['Loans Outstanding'],
            'Cash Reserves': balances['Cash Reserves'],
            'NOSTRO Account': balances['NOSTRO Account'],
            'Total Assets': balances['Loans Outstanding'] + balances['Cash Reserves'] + balances['NOSTRO Account']
        },
        'Liabilities': {
            'Customer Deposits': balances['Customer Deposits'],
            'Total Liabilities': balances['Customer Deposits']
        },
        'Equity': {
            'Total Equity': balances['Customer Deposits'] + balances['Loans Outstanding'] + balances['Cash Reserves'] + balances['NOSTRO Account']
        }
    }
    return balance_sheet

# Save the ledger to SQLite database and CSV file
def save_ledger_to_db_and_csv(ledger, db_name="bank_ledger.db", csv_name="bank_ledger.csv"):
    df = pd.DataFrame(ledger, columns=['Date', 'Transaction Type', 'Amount', 'Balance After'])
    conn = sqlite3.connect(db_name)
    df.to_sql('ledger', conn, if_exists='replace', index=False)
    df.to_csv(csv_name, index=False)
    conn.close()

# Simulate a month of operations
def simulate_month(balances):
    all_ledgers = []
    
    # Start date for the month simulation
    start_date = datetime.now()
    
    # Run daily simulation for 30 days
    for day in range(30):
        date = (start_date + timedelta(days=day)).strftime('%Y-%m-%d')
        daily_ledger = simulate_day(balances)
        
        # Append each transaction with the date to all_ledgers
        for transaction in daily_ledger:
            all_ledgers.append([date] + transaction)
        
        # Generate and print the daily financial report
        daily_report = generate_daily_report(balances)
        print(f"Day {day+1} Financial Report: {date}")
        print(pd.DataFrame(daily_report).T)
    
    return all_ledgers

# Main execution function
def main():
    balances = initialize_balances()
    
    # Simulate for one day
    print("\nSimulating one day of banking operations...")
    one_day_ledger = simulate_day(balances)
    print(one_day_ledger)
    
    # Simulate for a month
    print("\nSimulating one month of banking operations...")
    monthly_ledger = simulate_month(balances)
    
    # Save the ledger to database and CSV file
    print("\nSaving ledger to database and CSV...")
    save_ledger_to_db_and_csv(monthly_ledger)
    print("Ledger saved successfully!")

if __name__ == '__main__':
    main()



Simulating one day of banking operations...
[['Deposit', 11671, 5011671], ['Loan Disbursed', 61819, 2061819], ['Client Payment', 8710, 5002961], ['Treasury Transaction', 106974, 393026], ['Interest Income', 10630, 1055785]]

Simulating one month of banking operations...
Day 1 Financial Report: 2024-09-15
             Loans Outstanding  Cash Reserves  NOSTRO Account  Total Assets  \
Assets               2217387.0       967861.0        337904.0     3523152.0   
Liabilities                NaN            NaN             NaN           NaN   
Equity                     NaN            NaN             NaN           NaN   

             Customer Deposits  Total Liabilities  Total Equity  
Assets                     NaN                NaN           NaN  
Liabilities          5007341.0          5007341.0           NaN  
Equity                     NaN                NaN     8530493.0  
Day 2 Financial Report: 2024-09-16
             Loans Outstanding  Cash Reserves  NOSTRO Account  Total Assets  

# 2

To incorporate a 10% fee for each transaction, either for a loan or deposit, we need to adjust the code to deduct the fee from the client’s account and add it to a new account, say **Fee Income**, which will track the bank’s income from these transaction fees.

Here’s what we will modify:

1. For **Deposits**:
   - Charge a 10% fee on the deposit.
   - Deduct the fee from the customer’s deposit.
   - Record the fee as **Fee Income** (Credit).
   - Record the remaining deposit as a normal deposit transaction.

2. For **Loans**:
   - Charge a 10% fee on the loan disbursed.
   - Deduct the fee from the disbursed loan amount.
   - Record the fee as **Fee Income** (Credit).
   - Record the remaining loan amount in the customer’s loan account.

### Updated Python Code:

```python
import pandas as pd
import random
import sqlite3
from datetime import datetime, timedelta

# Initialize the bank's starting balances
def initialize_balances():
    return {
        'Customer Deposits': 5000000,
        'Loans Outstanding': 2000000,
        'Cash Reserves': 1000000,
        'NOSTRO Account': 500000,
        'Fee Income': 0  # New account for Fee Income
    }

# Helper function to calculate 10% transaction fee
def apply_fee(amount):
    return amount * 0.10

# Daily transactions simulation with debit and credit tags, including 10% fee
def simulate_day(balances):
    # Initialize daily ledger
    daily_ledger = []
    
    # Randomize deposit, loans, payments, and treasury transactions
    deposit = random.randint(10000, 100000)
    loan_disbursed = random.randint(50000, 250000)
    payment_processed = random.randint(5000, 30000)
    treasury_transaction = random.randint(50000, 150000)
    
    # 10% Fee for Deposit transaction
    deposit_fee = apply_fee(deposit)
    net_deposit = deposit - deposit_fee
    
    # Deposit transaction (increase Customer Deposits - Cr, increase Cash Reserves - Dr)
    balances['Customer Deposits'] += net_deposit
    balances['Cash Reserves'] += net_deposit
    balances['Fee Income'] += deposit_fee
    daily_ledger.append(['Deposit', 'Credit', net_deposit, 'Customer Deposits', balances['Customer Deposits']])
    daily_ledger.append(['Deposit', 'Debit', net_deposit, 'Cash Reserves', balances['Cash Reserves']])
    daily_ledger.append(['Deposit Fee', 'Credit', deposit_fee, 'Fee Income', balances['Fee Income']])
    
    # 10% Fee for Loan Disbursed
    loan_fee = apply_fee(loan_disbursed)
    net_loan_disbursed = loan_disbursed - loan_fee
    
    # Loan disbursed (increase Loans Outstanding - Dr, decrease Cash Reserves - Cr)
    if balances['Cash Reserves'] >= net_loan_disbursed:
        balances['Loans Outstanding'] += net_loan_disbursed
        balances['Cash Reserves'] -= net_loan_disbursed
        balances['Fee Income'] += loan_fee
        daily_ledger.append(['Loan Disbursed', 'Debit', net_loan_disbursed, 'Loans Outstanding', balances['Loans Outstanding']])
        daily_ledger.append(['Loan Disbursed', 'Credit', net_loan_disbursed, 'Cash Reserves', balances['Cash Reserves']])
        daily_ledger.append(['Loan Fee', 'Credit', loan_fee, 'Fee Income', balances['Fee Income']])
    
    # Payment processed for clients (decrease Customer Deposits - Dr, decrease Cash Reserves - Cr)
    if balances['Customer Deposits'] >= payment_processed:
        balances['Customer Deposits'] -= payment_processed
        balances['Cash Reserves'] -= payment_processed
        daily_ledger.append(['Client Payment', 'Debit', payment_processed, 'Customer Deposits', balances['Customer Deposits']])
        daily_ledger.append(['Client Payment', 'Credit', payment_processed, 'Cash Reserves', balances['Cash Reserves']])
    
    # Treasury transaction (NOSTRO account decrease - Cr, Cash Reserves increase - Dr)
    if balances['NOSTRO Account'] >= treasury_transaction:
        balances['NOSTRO Account'] -= treasury_transaction
        balances['Cash Reserves'] += treasury_transaction
        daily_ledger.append(['Treasury Transaction', 'Credit', treasury_transaction, 'NOSTRO Account', balances['NOSTRO Account']])
        daily_ledger.append(['Treasury Transaction', 'Debit', treasury_transaction, 'Cash Reserves', balances['Cash Reserves']])
    
    # Calculate daily interest income (increase Cash Reserves - Dr, increase Interest Income - Cr)
    daily_interest_income = random.randint(5000, 15000)  # Interest income on loans
    balances['Cash Reserves'] += daily_interest_income
    daily_ledger.append(['Interest Income', 'Debit', daily_interest_income, 'Cash Reserves', balances['Cash Reserves']])
    daily_ledger.append(['Interest Income', 'Credit', daily_interest_income, 'Interest Income', None])
    
    return daily_ledger

# Generate the daily financial report
def generate_daily_report(balances):
    balance_sheet = {
        'Assets': {
            'Loans Outstanding': balances['Loans Outstanding'],
            'Cash Reserves': balances['Cash Reserves'],
            'NOSTRO Account': balances['NOSTRO Account'],
            'Total Assets': balances['Loans Outstanding'] + balances['Cash Reserves'] + balances['NOSTRO Account']
        },
        'Liabilities': {
            'Customer Deposits': balances['Customer Deposits'],
            'Total Liabilities': balances['Customer Deposits']
        },
        'Equity': {
            'Fee Income': balances['Fee Income'],
            'Total Equity': balances['Fee Income'] + balances['Customer Deposits'] + balances['Loans Outstanding'] + balances['Cash Reserves'] + balances['NOSTRO Account']
        }
    }
    return balance_sheet

# Save the ledger to SQLite database and CSV file
def save_ledger_to_db_and_csv(ledger, db_name="bank_ledger.db", csv_name="bank_ledger.csv"):
    df = pd.DataFrame(ledger, columns=['Transaction Type', 'Dr/Cr', 'Amount', 'Account', 'Balance After'])
    conn = sqlite3.connect(db_name)
    df.to_sql('ledger', conn, if_exists='replace', index=False)
    df.to_csv(csv_name, index=False)
    conn.close()

# Simulate a month of operations
def simulate_month(balances):
    all_ledgers = []
    
    # Start date for the month simulation
    start_date = datetime.now()
    
    # Run daily simulation for 30 days
    for day in range(30):
        date = (start_date + timedelta(days=day)).strftime('%Y-%m-%d')
        daily_ledger = simulate_day(balances)
        
        # Append each transaction with the date to all_ledgers
        for transaction in daily_ledger:
            all_ledgers.append([date] + transaction)
        
        # Generate and print the daily financial report
        daily_report = generate_daily_report(balances)
        print(f"Day {day+1} Financial Report: {date}")
        print(pd.DataFrame(daily_report).T)
    
    return all_ledgers

# Main execution function
def main():
    balances = initialize_balances()
    
    # Simulate for one day
    print("\nSimulating one day of banking operations with 10% transaction fee...")
    one_day_ledger = simulate_day(balances)
    print(pd.DataFrame(one_day_ledger, columns=['Transaction Type', 'Dr/Cr', 'Amount', 'Account', 'Balance After']))
    
    # Simulate for a month
    print("\nSimulating one month of banking operations...")
    monthly_ledger = simulate_month(balances)
    
    # Save the ledger to database and CSV file
    print("\nSaving ledger to database and CSV...")
    save_ledger_to_db_and_csv(monthly_ledger)
    print("Ledger saved successfully!")

if __name__ == '__main__':
    main()
```

### Key Changes:
1. **Transaction Fee Calculation**:
   - The `apply_fee` function calculates a 10% fee on each deposit and loan.
   - The net amount (after deducting the fee) is used in the transactions.
   - The fee is credited to a new account called **Fee Income**.

2. **Deposit Fee**:
   - For every deposit, the customer is charged a 10% fee, which is credited to the **Fee Income** account, while the rest of the deposit is processed normally.

3. **Loan Fee**:
   - Similarly, for every loan disbursement, a 10% fee is charged and credited to **Fee Income**, while the rest of the loan is disbursed to the customer.

4. **Fee Income Tracking**:
   - The `Fee Income` account is updated to track the total fees earned by the bank from both deposits and loans.

5. **Updated Financial Report**:
   - The **Fee Income** account is now part of the bank's equity, and the daily financial report shows the accrued fee income.

### Expected Output:
- **Ledger**: The output ledger will include an entry for the fee charged on both deposits and loans.
- **Financial Reports**: The daily and monthly financial reports will show the fee income generated by the bank as part of its equity.
- **Database and CSV

**: The ledger will be saved to both an SQLite database (`bank_ledger.db`) and a CSV file (`bank_ledger.csv`).

This version ensures that clients are charged a 10% fee for each transaction and that the bank’s fee income is tracked properly in the ledger and financial reports.

The error you’re encountering, "5 columns passed, passed data had 6 columns," is caused by a mismatch between the number of columns specified in `pd.DataFrame` and the actual data in `daily_ledger`. The `daily_ledger` list contains more columns than expected when creating the DataFrame.

Here’s what’s happening:
- In the `daily_ledger` list, each transaction entry contains **6 elements** (date + 5 columns), but when creating the DataFrame, you are specifying only **5 columns**.

### Solution:
To fix this, we need to ensure that when creating the DataFrame from the `all_ledgers` list, the correct number of columns is passed.

Here’s how to fix it:

1. **Modify the column names**: Since `date` is appended along with the transaction details, we need to account for it as an additional column.
2. **Adjust the DataFrame creation**: Ensure that when converting `all_ledgers` to a DataFrame, we provide the correct number of column names.

### Updated Code:

```python
import pandas as pd
import random
import sqlite3
from datetime import datetime, timedelta

# Initialize the bank's starting balances
def initialize_balances():
    return {
        'Customer Deposits': 5000000,
        'Loans Outstanding': 2000000,
        'Cash Reserves': 1000000,
        'NOSTRO Account': 500000,
        'Fee Income': 0  # New account for Fee Income
    }

# Helper function to calculate 10% transaction fee
def apply_fee(amount):
    return amount * 0.10

# Daily transactions simulation with debit and credit tags, including 10% fee
def simulate_day(balances):
    # Initialize daily ledger
    daily_ledger = []
    
    # Randomize deposit, loans, payments, and treasury transactions
    deposit = random.randint(10000, 100000)
    loan_disbursed = random.randint(50000, 250000)
    payment_processed = random.randint(5000, 30000)
    treasury_transaction = random.randint(50000, 150000)
    
    # 10% Fee for Deposit transaction
    deposit_fee = apply_fee(deposit)
    net_deposit = deposit - deposit_fee
    
    # Deposit transaction (increase Customer Deposits - Cr, increase Cash Reserves - Dr)
    balances['Customer Deposits'] += net_deposit
    balances['Cash Reserves'] += net_deposit
    balances['Fee Income'] += deposit_fee
    daily_ledger.append(['Deposit', 'Credit', net_deposit, 'Customer Deposits', balances['Customer Deposits']])
    daily_ledger.append(['Deposit', 'Debit', net_deposit, 'Cash Reserves', balances['Cash Reserves']])
    daily_ledger.append(['Deposit Fee', 'Credit', deposit_fee, 'Fee Income', balances['Fee Income']])
    
    # 10% Fee for Loan Disbursed
    loan_fee = apply_fee(loan_disbursed)
    net_loan_disbursed = loan_disbursed - loan_fee
    
    # Loan disbursed (increase Loans Outstanding - Dr, decrease Cash Reserves - Cr)
    if balances['Cash Reserves'] >= net_loan_disbursed:
        balances['Loans Outstanding'] += net_loan_disbursed
        balances['Cash Reserves'] -= net_loan_disbursed
        balances['Fee Income'] += loan_fee
        daily_ledger.append(['Loan Disbursed', 'Debit', net_loan_disbursed, 'Loans Outstanding', balances['Loans Outstanding']])
        daily_ledger.append(['Loan Disbursed', 'Credit', net_loan_disbursed, 'Cash Reserves', balances['Cash Reserves']])
        daily_ledger.append(['Loan Fee', 'Credit', loan_fee, 'Fee Income', balances['Fee Income']])
    
    # Payment processed for clients (decrease Customer Deposits - Dr, decrease Cash Reserves - Cr)
    if balances['Customer Deposits'] >= payment_processed:
        balances['Customer Deposits'] -= payment_processed
        balances['Cash Reserves'] -= payment_processed
        daily_ledger.append(['Client Payment', 'Debit', payment_processed, 'Customer Deposits', balances['Customer Deposits']])
        daily_ledger.append(['Client Payment', 'Credit', payment_processed, 'Cash Reserves', balances['Cash Reserves']])
    
    # Treasury transaction (NOSTRO account decrease - Cr, Cash Reserves increase - Dr)
    if balances['NOSTRO Account'] >= treasury_transaction:
        balances['NOSTRO Account'] -= treasury_transaction
        balances['Cash Reserves'] += treasury_transaction
        daily_ledger.append(['Treasury Transaction', 'Credit', treasury_transaction, 'NOSTRO Account', balances['NOSTRO Account']])
        daily_ledger.append(['Treasury Transaction', 'Debit', treasury_transaction, 'Cash Reserves', balances['Cash Reserves']])
    
    # Calculate daily interest income (increase Cash Reserves - Dr, increase Interest Income - Cr)
    daily_interest_income = random.randint(5000, 15000)  # Interest income on loans
    balances['Cash Reserves'] += daily_interest_income
    daily_ledger.append(['Interest Income', 'Debit', daily_interest_income, 'Cash Reserves', balances['Cash Reserves']])
    daily_ledger.append(['Interest Income', 'Credit', daily_interest_income, 'Interest Income', None])
    
    return daily_ledger

# Generate the daily financial report
def generate_daily_report(balances):
    balance_sheet = {
        'Assets': {
            'Loans Outstanding': balances['Loans Outstanding'],
            'Cash Reserves': balances['Cash Reserves'],
            'NOSTRO Account': balances['NOSTRO Account'],
            'Total Assets': balances['Loans Outstanding'] + balances['Cash Reserves'] + balances['NOSTRO Account']
        },
        'Liabilities': {
            'Customer Deposits': balances['Customer Deposits'],
            'Total Liabilities': balances['Customer Deposits']
        },
        'Equity': {
            'Fee Income': balances['Fee Income'],
            'Total Equity': balances['Fee Income'] + balances['Customer Deposits'] + balances['Loans Outstanding'] + balances['Cash Reserves'] + balances['NOSTRO Account']
        }
    }
    print("\nDaily Financial Report:")
    print(pd.DataFrame(balance_sheet).T)
    return balance_sheet

# Save the ledger to SQLite database and CSV file
def save_ledger_to_db_and_csv(ledger, db_name="bank_ledger.db", csv_name="bank_ledger.csv"):
    df = pd.DataFrame(ledger, columns=['Date', 'Transaction Type', 'Dr/Cr', 'Amount', 'Account', 'Balance After'])
    conn = sqlite3.connect(db_name)
    df.to_sql('ledger', conn, if_exists='replace', index=False)
    df.to_csv(csv_name, index=False)
    conn.close()

# Simulate a month of operations
def simulate_month(balances):
    all_ledgers = []
    
    # Start date for the month simulation
    start_date = datetime.now()
    
    # Run daily simulation for 30 days
    for day in range(30):
        date = (start_date + timedelta(days=day)).strftime('%Y-%m-%d')
        daily_ledger = simulate_day(balances)
        
        # Append each transaction with the date to all_ledgers
        for transaction in daily_ledger:
            all_ledgers.append([date] + transaction)
        
        # Generate and print the daily financial report
        print(f"\n=== Day {day+1} Financial Report for {date} ===")
        generate_daily_report(balances)
    
    return all_ledgers

# Main execution function
def main():
    balances = initialize_balances()
    
    # Simulate for one day
    print("\nSimulating one day of banking operations with 10% transaction fee...")
    one_day_ledger = simulate_day(balances)
    print(pd.DataFrame(one_day_ledger, columns=['Transaction Type', 'Dr/Cr', 'Amount', 'Account', 'Balance After']))
    
    # Simulate for a month
    print("\nSimulating one month of banking operations...")
    monthly_ledger = simulate_month(balances)
    
    # Save the ledger to database and CSV file
    print("\nSaving ledger to database and CSV...")
    save_ledger_to_db_and_csv(monthly_ledger)
    print("Ledger saved successfully!")

if __name__ == '__main__':
    main()
```

### Explanation of Fixes:
- **Correct number of columns**: When creating the DataFrame, I updated the columns to include `Date` as the first column, followed by the other transaction details (`Transaction Type`, `Dr/Cr`, `Amount`, `Account`, `Balance After`).
- **Appending Date**: In the `simulate_month` function, I ensured that the `date` is appended as the first column in each transaction, resulting in the correct number of columns.

This should resolve the mismatch error, and the ledger will now have the correct number of columns when saved to the database or CSV.

# 2 code

In [6]:
import pandas as pd
import random
import sqlite3
from datetime import datetime, timedelta

# Initialize the bank's starting balances
def initialize_balances():
    return {
        'Customer Deposits': 5000000,
        'Loans Outstanding': 2000000,
        'Cash Reserves': 1000000,
        'NOSTRO Account': 500000,
        'Fee Income': 0  # New account for Fee Income
    }

# Helper function to calculate 10% transaction fee
def apply_fee(amount):
    return amount * 0.10

# Daily transactions simulation with debit and credit tags, including 10% fee
def simulate_day(balances):
    # Initialize daily ledger
    daily_ledger = []
    
    # Randomize deposit, loans, payments, and treasury transactions
    deposit = random.randint(10000, 100000)
    loan_disbursed = random.randint(50000, 250000)
    payment_processed = random.randint(5000, 30000)
    treasury_transaction = random.randint(50000, 150000)
    
    # 10% Fee for Deposit transaction
    deposit_fee = apply_fee(deposit)
    net_deposit = deposit - deposit_fee
    
    # Deposit transaction (increase Customer Deposits - Cr, increase Cash Reserves - Dr)
    balances['Customer Deposits'] += net_deposit
    balances['Cash Reserves'] += net_deposit
    balances['Fee Income'] += deposit_fee
    daily_ledger.append(['Deposit', 'Credit', net_deposit, 'Customer Deposits', balances['Customer Deposits']])
    daily_ledger.append(['Deposit', 'Debit', net_deposit, 'Cash Reserves', balances['Cash Reserves']])
    daily_ledger.append(['Deposit Fee', 'Credit', deposit_fee, 'Fee Income', balances['Fee Income']])
    
    # 10% Fee for Loan Disbursed
    loan_fee = apply_fee(loan_disbursed)
    net_loan_disbursed = loan_disbursed - loan_fee
    
    # Loan disbursed (increase Loans Outstanding - Dr, decrease Cash Reserves - Cr)
    if balances['Cash Reserves'] >= net_loan_disbursed:
        balances['Loans Outstanding'] += net_loan_disbursed
        balances['Cash Reserves'] -= net_loan_disbursed
        balances['Fee Income'] += loan_fee
        daily_ledger.append(['Loan Disbursed', 'Debit', net_loan_disbursed, 'Loans Outstanding', balances['Loans Outstanding']])
        daily_ledger.append(['Loan Disbursed', 'Credit', net_loan_disbursed, 'Cash Reserves', balances['Cash Reserves']])
        daily_ledger.append(['Loan Fee', 'Credit', loan_fee, 'Fee Income', balances['Fee Income']])
    
    # Payment processed for clients (decrease Customer Deposits - Dr, decrease Cash Reserves - Cr)
    if balances['Customer Deposits'] >= payment_processed:
        balances['Customer Deposits'] -= payment_processed
        balances['Cash Reserves'] -= payment_processed
        daily_ledger.append(['Client Payment', 'Debit', payment_processed, 'Customer Deposits', balances['Customer Deposits']])
        daily_ledger.append(['Client Payment', 'Credit', payment_processed, 'Cash Reserves', balances['Cash Reserves']])
    
    # Treasury transaction (NOSTRO account decrease - Cr, Cash Reserves increase - Dr)
    if balances['NOSTRO Account'] >= treasury_transaction:
        balances['NOSTRO Account'] -= treasury_transaction
        balances['Cash Reserves'] += treasury_transaction
        daily_ledger.append(['Treasury Transaction', 'Credit', treasury_transaction, 'NOSTRO Account', balances['NOSTRO Account']])
        daily_ledger.append(['Treasury Transaction', 'Debit', treasury_transaction, 'Cash Reserves', balances['Cash Reserves']])
    
    # Calculate daily interest income (increase Cash Reserves - Dr, increase Interest Income - Cr)
    daily_interest_income = random.randint(5000, 15000)  # Interest income on loans
    balances['Cash Reserves'] += daily_interest_income
    daily_ledger.append(['Interest Income', 'Debit', daily_interest_income, 'Cash Reserves', balances['Cash Reserves']])
    daily_ledger.append(['Interest Income', 'Credit', daily_interest_income, 'Interest Income', None])
    
    return daily_ledger

# Generate the daily financial report
def generate_daily_report(balances):
    balance_sheet = {
        'Assets': {
            'Loans Outstanding': balances['Loans Outstanding'],
            'Cash Reserves': balances['Cash Reserves'],
            'NOSTRO Account': balances['NOSTRO Account'],
            'Total Assets': balances['Loans Outstanding'] + balances['Cash Reserves'] + balances['NOSTRO Account']
        },
        'Liabilities': {
            'Customer Deposits': balances['Customer Deposits'],
            'Total Liabilities': balances['Customer Deposits']
        },
        'Equity': {
            'Fee Income': balances['Fee Income'],
            'Total Equity': balances['Fee Income'] + balances['Customer Deposits'] + balances['Loans Outstanding'] + balances['Cash Reserves'] + balances['NOSTRO Account']
        }
    }
    return balance_sheet

# Save the ledger and financial report to SQLite database and CSV files
def save_data_to_db_and_csv(ledger, reports, db_name="bank_ledger.db", ledger_csv="bank_ledger.csv", report_csv="financial_report.csv"):
    # Save the ledger
    df_ledger = pd.DataFrame(ledger, columns=['Date', 'Transaction Type', 'Dr/Cr', 'Amount', 'Account', 'Balance After'])
    conn = sqlite3.connect(db_name)
    df_ledger.to_sql('ledger', conn, if_exists='replace', index=False)
    df_ledger.to_csv(ledger_csv, index=False)

    # Save the financial reports
    df_reports = pd.DataFrame(reports)
    df_reports.to_csv(report_csv, index=False)

    conn.close()

# Simulate a month of operations
def simulate_month(balances):
    all_ledgers = []
    all_reports = []
    
    # Start date for the month simulation
    start_date = datetime.now()
    
    # Run daily simulation for 30 days
    for day in range(30):
        date = (start_date + timedelta(days=day)).strftime('%Y-%m-%d')
        daily_ledger = simulate_day(balances)
        
        # Append each transaction with the date to all_ledgers
        for transaction in daily_ledger:
            all_ledgers.append([date] + transaction)
        
        # Generate daily financial report
        daily_report = generate_daily_report(balances)
        daily_report['Date'] = date
        all_reports.append(daily_report)
        
        # Print daily financial report (optional)
        print(f"\n=== Day {day+1} Financial Report for {date} ===")
        print(pd.DataFrame(daily_report).T)
    
    return all_ledgers, all_reports

# Main execution function
def main():
    balances = initialize_balances()
    
    # Simulate for a month
    print("\nSimulating one month of banking operations...")
    monthly_ledger, monthly_reports = simulate_month(balances)
    
    # Save the ledger and financial reports to database and CSV files
    print("\nSaving ledger and financial reports to database and CSV files...")
    save_data_to_db_and_csv(monthly_ledger, monthly_reports)
    print("Ledger and financial reports saved successfully!")

if __name__ == '__main__':
    main()



Simulating one month of banking operations...

=== Day 1 Financial Report for 2024-09-15 ===
            Loans Outstanding Cash Reserves NOSTRO Account Total Assets  \
Assets              2220168.8      905559.1       423920.0    3549647.9   
Liabilities               NaN           NaN            NaN          NaN   
Equity                    NaN           NaN            NaN          NaN   
Date               2024-09-15    2024-09-15     2024-09-15   2024-09-15   

            Customer Deposits Total Liabilities  Fee Income Total Equity  
Assets                    NaN               NaN         NaN          NaN  
Liabilities         5044140.9         5044140.9         NaN          NaN  
Equity                    NaN               NaN     30516.3    8624305.1  
Date               2024-09-15        2024-09-15  2024-09-15   2024-09-15  

=== Day 2 Financial Report for 2024-09-16 ===
            Loans Outstanding Cash Reserves NOSTRO Account Total Assets  \
Assets              2378036.0    